In [1]:
import os
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

import torch
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, models
from tqdm.notebook import tqdm, trange



def torch_stats(): 
    torch_version = ".".join(torch.__version__.split(".")[:2])
    print('torch version:',torch_version)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
    dtype = torch.float32
    
    if device.type == 'cuda':
        cuda_version  = torch.__version__.split("+")[-1]
        print("cuda: ", cuda_version)
        
        torch.set_default_tensor_type(torch.cuda.FloatTensor)
        print('Cuda is available:',torch.cuda.is_available())

        n_devices = torch.cuda.device_count()
        print('number of devices: %d'%(n_devices))
        if n_devices > 0:
            device = 'cuda:0'
        for cnt_device in range(n_devices):
            print(torch.cuda.get_device_name(cnt_device))
            print('Memory Usage:')
            print('Allocated:', round(torch.cuda.memory_allocated(cnt_device)/1024**3,1), 'GB')
            print('Cached:   ', round(torch.cuda.memory_reserved(cnt_device)/1024**3,1), 'GB')


    torch.set_default_dtype(dtype) # float32
    print('default data type:',dtype)
    
    num_workers=os.cpu_count()
    print ('available number of workers:',num_workers)
    
    return device, dtype, num_workers
#-------------------------------
def torch_seed(seed = 42, deterministic = True):
    random.seed(seed) # random and transforms
    np.random.seed(seed) #numpy
    torch.manual_seed(seed) #cpu
    torch.cuda.manual_seed(seed) #gpu
    torch.backends.cudnn.deterministic=deterministic #cudnn 



device, dtype, num_workers = torch_stats()
torch_seed(seed = 42, deterministic = True)

from torchvision.datasets.utils import download_and_extract_archive
url = 'https://github.com/MVRonkin/Deep-Learning-Foundation-Course/raw/main/2024Light/ContestDataset.zip'
root_directory = os.path.join(os.getcwd(),'.')
download_and_extract_archive(url, root_directory)

torch version: 2.6
Using device: cuda
cuda:  cu124
Cuda is available: True
number of devices: 1
NVIDIA GeForce RTX 4070
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
default data type: torch.float32
available number of workers: 24


/home/balots/jupyter-venv/lib/python3.12/site-packages/torch/__init__.py:1236: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [2]:
SIZE = (32,32)

#transform_ = lambda x: x.flatten()/255

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

path2data = "./ContestDataset/"
os.listdir(path2data)

['train', 'test.csv', 'test_open', 'test', 'train.csv', 'test_open.csv']

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
csv = pd.read_csv(os.path.join(path2data,'test_open.csv'))
print(np.unique(le.fit_transform(csv['class'])), csv['class'].unique() )

[0 1 2 3 4] ['airplane' 'automobile' 'bird' 'cat' 'deer']


In [4]:
class ContestDataset(Dataset):
    def __init__(self, image_dir='train', data_path='train.csv', transform=None, le=False):
        self.image_dir = image_dir
        self.transform = transform
        self.le = le

        csv = pd.read_csv(data_path)

        # --- Удаляем дубликаты и сбрасываем индекс ---
        csv = csv.drop_duplicates(subset=['file_name'], keep='first').reset_index(drop=True)

        self.fnames = csv['file_name']

        if 'class' in csv.columns:
            if self.le == False:
                self.le = LabelEncoder()
                self.le.fit(csv['class'])

            self.class_names = list(csv['class'].unique())
            self.labels = self.le.transform(csv['class'])
            self.labels_exist = True
        else:
            self.labels = None
            self.labels_exist = False

    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.fnames.iloc[idx])
        img = Image.open(img_path)
    
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)
    
        if self.labels_exist:
            label = self.labels[idx]
        else:
            label = self.fnames.iloc[idx]
    
        return img, label

In [5]:
train_data =  ContestDataset(image_dir = os.path.join(path2data,'train'), 
                             data_path = os.path.join(path2data,'train.csv'),                              
                             transform=transform, le = le)

test_data =  ContestDataset(image_dir = os.path.join(path2data,'test_open'), 
                            data_path = os.path.join(path2data,'test_open.csv'), 
                            transform=transform, le = le)

submit_data =  ContestDataset(image_dir = os.path.join(path2data,'test'), 
                              data_path = os.path.join(path2data,'test.csv'), 
                              transform=transform, le = le)

In [6]:
g = torch.Generator(device)  # ← Always CPU for random_split
g.manual_seed(42)

In [7]:
TRAIN_RATIO = 0.8

n_train_examples = int(len(train_data) * TRAIN_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_valid_examples], generator=g)

print(f"Training data size : {len(train_data)}, Validation data size : {len(valid_data)}, Test data size : {len(test_data)}")

Training data size : 3896, Validation data size : 974, Test data size : 1970


In [8]:
import torch.utils.data as data

BATCH_SIZE = 64

train_loader = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE,
                                 generator=g)

val_loader = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE,
                                 generator=g)

test_loader = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                             generator=g)


submit_loader = data.DataLoader(submit_data,
                                batch_size=BATCH_SIZE,
                               generator=g)
 
print(f"Training data batches : {len(train_loader)}, Validation data batches : {len(val_loader)}, Test data batches : {len(test_loader)}, Submit data batches : {len(test_loader)}")

Training data batches : 61, Validation data batches : 16, Test data batches : 31, Submit data batches : 31


In [9]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        
        self.conv_layers = nn.Sequential(
            # [1, 32, 32] → [32, 32, 32]
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),       # [32, 16, 16]

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),       # [64, 8, 8]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),       # [128, 4, 4]
            nn.Dropout(0.3),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # Вход из DataLoader теперь (batch, 1, 32, 32)
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)
        x = self.fc_layers(x)
        return x


In [10]:
def train(model, dataloader, optimizer, criterion, metric,  device):

    epoch_loss = 0
    epoch_acc  = 0

    model.train()

    for (x, y) in tqdm(dataloader, desc="Training", leave=False):

        x, y = x.to(device).float(), y.to(device).long()

        optimizer.zero_grad(set_to_none = True)

        y_pred = model(x)

        loss = criterion(y_pred, y)
        acc  = metric(y_pred, y)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc  += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

def evaluate(model, dataloader, criterion, metric, device):

    epoch_loss = 0
    epoch_acc  = 0

    model.eval()

    with torch.inference_mode():
        
        for (x, y) in tqdm(dataloader, desc="Evaluating", leave=False):

            x, y = x.to(device).float(), y.to(device).long()

            y_pred = model.forward(x)
            
            loss = criterion(y_pred, y)
            acc  = metric( y_pred, y)

            epoch_loss += loss.item()
            epoch_acc  += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

EPOCHS = 100

def fit(model, train_loader, val_loader, optimizer, criterion, metric, epochs = EPOCHS, device='cpu',  path_best = 'best_model.pt', verbose = True):

   
    best_valid_loss = float('inf')

    for epoch in trange(epochs):

        start_time = time.monotonic()

        train_loss, train_acc = train(model, train_loader, optimizer, criterion, metric, device)
        valid_loss, valid_acc = evaluate(model, val_loader, criterion, metric, device)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), path_best)

        end_time = time.monotonic()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if verbose == True:
            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%',
            f' | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%') 

In [12]:
model = CNNModel(num_classes=5).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

def calculate_accuracy(y_pred, y):
    with torch.no_grad():
        top_pred = y_pred.argmax(1, keepdim=True)
        correct = top_pred.eq(y.view_as(top_pred)).sum()
        acc = correct.float() / y.shape[0]
    return acc

metric = calculate_accuracy
fit(model, train_loader, val_loader, optimizer, criterion, metric,
    epochs = 50, device=device, verbose = True)

model.load_state_dict(torch.load('best_model.pt'))

test_loss, test_acc = evaluate(model, test_loader, criterion, metric, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

  0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/61 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.423 | Train Acc: 38.42%  | Val. Loss: 1.233 | Val. Acc: 51.87%


Training:   0%|          | 0/61 [00:00<?, ?it/s]

...

Training:   0%|          | 0/61 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 50 | Epoch Time: 0m 1s
	Train Loss: 0.579 | Train Acc: 76.92%  | Val. Loss: 0.909 | Val. Acc: 70.88%


Evaluating:   0%|          | 0/31 [00:00<?, ?it/s]

Test Loss: 0.972 | Test Acc: 62.72%


In [13]:
def submit(model, dataloader, device):

    df = pd.DataFrame(columns = ('file_name','class'))
    model.eval()

    with torch.inference_mode():
        
        for x,y in tqdm(dataloader, desc="Evaluating", leave=False):

            x = x.to(device).float()
            y_pred = model.forward(x)
            if y_pred.is_cuda: 
                cls = list(y_pred.argmax(1, keepdim=False).data.cpu().numpy())
            else:
                cls = list(y_pred.argmax(1, keepdim=False).data.numpy())
            cls_label = le.inverse_transform(np.array((cls)))[0]
            df = pd.concat([df,pd.DataFrame({'file_name':y,'class':cls_label})], ignore_index = True)

    return df.reset_index()

len(submit_data)


df = submit(model, submit_loader, device)

df

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

,index,file_name,class
0,0,cifar10-train-25538.png,airplane
1,1,cifar10-train-1086.png,airplane
2,2,cifar10-train-12192.png,airplane
3,3,cifar10-train-43783.png,airplane
4,4,n03596543_2495.png,airplane
...,...,...,...
2419,2419,n02419796_10865.png,bird
2420,2420,n02430045_5648.png,bird
2421,2421,cifar10-train-2896.png,bird
2422,2422,cifar10-test-1945.png,bird


In [14]:
path_submit = 'submit.csv'

df[['file_name','class']].to_csv(path_submit, index = False )
print('FULL PATH:  \n', os.path.join(os.getcwd(), path_submit) )

FULL PATH:  
 /home/balots/PycharmProjects/train/ComputerVision/submit.csv


In [15]:
frame = pd.read_csv('submit.csv')

In [16]:


frame.groupby('class').count()



,file_name
class,
airplane,704
automobile,448
bird,376
cat,448
deer,448


In [17]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  2424 non-null   object
 1   class      2424 non-null   object
dtypes: object(2)
memory usage: 38.0+ KB
